In [ ]:
########### generate prompts

In [5]:
!ls llm_data

graphsage_asin_Toys_and_Games.json.npy
hyperbiolic_asin_Toys_and_Games.json.npy
label_graphsage_asin_Toys_and_Games.json.npy
label_hyperbiolic_asin_Toys_and_Games.json.npy
llm_meta_Toys_and_Games.json
test_label_scomgnn_ordered_asin_Toys_and_Games.npy
test_scomgnn_ordered_asin_Toys_and_Games.npy


In [1]:
import numpy as np
# data_name = 'Cell_Phones_and_Accessories'
data_name = 'Grocery_and_Gourmet_Food'
# data_name = 'Electronics'
# data_name = "Home_and_Kitchen"
# data_name = 'Toys_and_Games'
import json
with open('./llm_data/llm_meta_{}.json'.format(data_name), 'r') as file:
    data_feature = json.load(file)


In [2]:
# model = "sage"
model = "scomgnn"
# model = 'hyperbolic' 
# model = "decgcn" 
# model = "hgnn"
# model = "graphsage"
# model = "gat"
# model = "gin"
# data = np.load('./llm_data/test_anchor_ordered_asin_{}_{}.npy'.format(data_name, model))
data = np.load('./llm_data/test_anchor_ordered_asin_{}.npy'.format(data_name))

In [5]:
class Reranking():
    def __init__(self, dataset_name, data_id, data_feature, nodes="accuracy",
                 item_fea=None, top_k=10, focus="Overall Performance", max_count=3):
        self.nodes = nodes
        self.item_fea = item_fea
        self.top_k = top_k
        self.data_id = data_id
        self.data_feature = data_feature
        self.candidate_id = None
        self.focus = focus
        self.max_count = max_count
        self.dataset_name = dataset_name

    def diversity_rerank(self, asin):
        anchor = asin[0]
        candidate_items = asin[1:]
        current_name = 'diversity'
        prompt = "Considering a product, its basic infomation is: \n{"
        for fea in self.item_fea:
            prompt += "{}:{},".format(fea, str(self.data_feature[anchor][fea]))
        prompt += "}\nHere's a list of the candidate products: \n{"
        for i, item in enumerate(candidate_items):
            prompt += "["
            prompt += "{}:{},".format("ID", str(i))
            for fea in self.item_fea:
                prompt += "{}:{},".format(fea, str(self.data_feature[item][fea]))
            prompt += ("]\n")

        prompt += ("The task is identifing the complementary relation between the given product and candidates.\n"
                   "Complementary is defined as: products are likely to be purchased or used at the same time, but it is not a direct substitute.\n"
                   "The complementary product could be an accessory of the given product, for example, iPhone Case is the complementary of iPhone.\n"
                   "Or they are both accessories to the same product, for example, Speaker Cables can be the complementary of Speaker Stands.\n"
                   "Or they can be used together for the same activity, for example, Bowl can be the complementary of Plate.\n"
                    "Then rerank the candidates based on above given information.")
        
        prompt += (
                "\nThe order of reranking result should represent how likely the candidate is a complementary product."
                "\nMeanwhile, focus on the " + current_name + " aspect (more items with different 'genre' feature at the top of the list).")
        prompt += (
                "\nYour answer should ONLY rank all mentioned candidates ID, do NOT repeat or include Name."
                "And omit anything else such as your thinking and decision-making process."
                "\nExample answer format for 5 candidates: [1, 4, 3, 0, 2]")

        return prompt

    def accuracy_rerank(self, asin):
        anchor = asin[0]
        candidate_items = asin[1:]
        current_name = 'accuracy'
        prompt = "Considering a product, its basic infomation is: \n{"
        for fea in self.item_fea:
            prompt += "{}:{},".format(fea, str(self.data_feature[anchor][fea]))
        prompt += "}\nHere's a list of the candidate products: \n{"
        for i, item in enumerate(candidate_items):
            prompt += "["
            prompt += "{}:{},".format("ID", str(i))
            for fea in self.item_fea:
                prompt += "{}:{},".format(fea, str(self.data_feature[item][fea]))
            prompt += ("]\n")


        prompt += ("Now, you need to focus on the " + current_name + " of identifing the complementary relation between the given product and candidates.\n"
                   "Complementary is defined as: products are likely to be purchased or used at the same time, but it is not a direct substitute.\n"
                   "The complementary product could be an accessory of the given product, for example, iPhone Case is the complementary of iPhone.\n"
                   "Or they are both accessories to the same product, for example, Speaker Cables can be the complementary of Speaker Stands.\n"
                   "Or they can be used together for the same activity, for example, Bowl can be the complementary of Plate.\n"
                    "Then you will rerank the candidates based on the given information.")

        prompt += (
                "\nThe order of reranking result should represent how likely the candidate is a complementary product."
                "\nYour answer should ONLY rank all mentioned candidates ID, do NOT repeat or include Name."
                " And omit anything else such as your thinking and decision-making process."
                "\nExample answer format for 5 candidates: [1, 4, 3, 0, 2]")

        return prompt

    def rerank_batch(self, data_array, topk):
        """
        Process multiple rows of data at once.
        
        :param data_array: numpy array where each row represents a set of ASINs
        :return: list of prompts for each row
        """
        
        prompts = []
        for row in data_array:
            asin = row[:topk+1]  # Assuming you want to process the first 21 items of each row
            if self.nodes == "accuracy":
                prompt = self.accuracy_rerank(asin)
            elif self.nodes == "diversity":
                prompt = self.diversity_rerank(asin)
            prompts.append(prompt)
        return prompts



In [6]:
item_fea = ["title"]
rerank = Reranking(dataset_name = data_name, data_id = data, data_feature = data_feature, item_fea = item_fea)

In [7]:
prompt = rerank.rerank_batch(data, 50)


In [9]:
import boto3
def generate_sequential_id(index, length=11):
    """Generate a sequential ID padded with leading zeros."""
    return str(index).zfill(length)

json_objects = []

for index, p in enumerate(prompt):
    formatted_prompt = f"""
    <|begin_of_text|><|start_header_id|>user<|end_header_id|>
    {p}
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """
    json_object = {
        "recordId": generate_sequential_id(index),
        "modelInput": {
                "max_gen_len": 300,
                "prompt": formatted_prompt,
                "temperature": 0.1,
                "top_p": 0.1
                }
        }
    json_objects.append(json.dumps(json_object))
file_content = '\n'.join(json_objects)
s3 = boto3.client('s3')



In [10]:

bucket_name = 'llmrerank'
file_key = 'llama/input/pure_{}_accuracy_{}_50.jsonl'.format(model, data_name)

# Upload the file to S3
s3.put_object(Bucket=bucket_name, Key=file_key, Body=file_content)


{'ResponseMetadata': {'RequestId': 'E2ZG0S8NA46NCENP',
  'HostId': 'uddiKM5HIcLdhFMHXcqTJYkQiUNrB9JLdtr6NuLIuiULEt2SkhYCaBhh+iq+6rFs6cWom8JvzA9SkomPo+lc7Q==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'uddiKM5HIcLdhFMHXcqTJYkQiUNrB9JLdtr6NuLIuiULEt2SkhYCaBhh+iq+6rFs6cWom8JvzA9SkomPo+lc7Q==',
   'x-amz-request-id': 'E2ZG0S8NA46NCENP',
   'date': 'Wed, 16 Apr 2025 04:23:56 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"345316abfcf31d89c4612cf1ad636c6c"',
   'x-amz-checksum-crc32': 'kBRFFw==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 1},
 'ETag': '"345316abfcf31d89c4612cf1ad636c6c"',
 'ChecksumCRC32': 'kBRFFw==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}

In [183]:
import boto3
def generate_sequential_id(index, length=11):
    """Generate a sequential ID padded with leading zeros."""
    return str(index).zfill(length)


json_objects = []

for index, p in enumerate(prompt):
    json_object = {
        "recordId": generate_sequential_id(index),
        "modelInput": {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 300,
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": p
                        }
                    ]
                }
            ]
        }
    }
    json_objects.append(json.dumps(json_object))
file_content = '\n'.join(json_objects)
s3 = boto3.client('s3')



In [83]:
model, data_name

('scomgnn', 'Cell_Phones_and_Accessories')

In [84]:

bucket_name = 'llmrerank'
file_key = 'claude/input/prompt_{}_diversity_{}_100.jsonl'.format(model, data_name)

# Upload the file to S3
s3.put_object(Bucket=bucket_name, Key=file_key, Body=file_content)


{'ResponseMetadata': {'RequestId': 'HPEX414D454A14DA',
  'HostId': 'h8mVoUOlH8LfkYzHTRp6UY6HOSMeW2J/9u2efOl3smvppGqbnsL61JSfdVJ40li9hjI6cb741gA=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'h8mVoUOlH8LfkYzHTRp6UY6HOSMeW2J/9u2efOl3smvppGqbnsL61JSfdVJ40li9hjI6cb741gA=',
   'x-amz-request-id': 'HPEX414D454A14DA',
   'date': 'Thu, 10 Apr 2025 04:42:00 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"26cdc664195d4459cb7c8143a1588063"',
   'x-amz-checksum-crc32': 'q8AMww==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 1},
 'ETag': '"26cdc664195d4459cb7c8143a1588063"',
 'ChecksumCRC32': 'q8AMww==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}